# Chain

**Chain**(체인)은 여러 컴포넌트(요소)를 정해진 순서대로 연결하여 **복잡한 AI 작업을 단계별로 자동화**할 수 있도록 돕는 구조이다.

- 각 컴포넌트는 **이전 처리결과를 입력으로 받아 처리한 후 다음 단계로 결과를 전달**한다.
- 복잡한 작업을 여러 개의 단순한 단계로 나누고, 각 단계를 순차적으로 실행함으로써 전체 작업을 체계적으로 구성할 수 있다.

## 기본 개념

- 체인은 하나의 LLM 호출에 그치지 않고 **여러 LLM 호출이나 도구 실행등을 순차적으로 연결**하여 실행 할 수 있다.
- 예를 들어, 사용자의 질문 → 검색 → 요약 → 응답 생성 같은 일련의 작업을 체인으로 구성할 수 있다.
- 이러한 체인구조를 사용하면 **작업흐름이 명확**해지고 **코드의 재사용성**이 높아지며 **유지 보수 및 확장성이** 향상된다.

## LangChain에서의 Chain 구성 방식

LangChain은 다음 두 가지 방식을 통해 체인을 구성할 수 있다.

### 1. Off-the-shelf Chains 방식 (클래식 방식)

- LangChain에서 제공하는 **미리 정의된 Chain 클래스**(예: `LLMChain`, `SequentialChain`, `SimpleSequentialChain`)를 활용하는 방식이다.
- 각 클래스는 다양한 chain 알고리즘들을 미리 구현한 것으로 상황에 맞는 것을 선택하여 필요한 구성요소를 전달해 생생한다.
- 이 방식은 LangChain의 **초기 방식**이며, 새로운 기능 확장이나 유연한 구성에 한계가 있기 때문에 현재 **더 이상 사용되지 않음(deprecated)** 상태이다.
  - 현재 LangChain에서는 권장하지 않는 방식이다.

### 2. LCEL (LangChain Expression Language) 방식

- LCEL은 체인을 표현식(Expression) 기반의 선언적 파이프라인 방식으로 구성할 수 있도록 설계된 최신 체인 구성 방법이다. 
- 각 컴포넌트들을 `|` 연산자로 연결하여, 흐름이 자연스럽게 이어지는 형태의 체인을 구성한다.
- LCEL 방식은 간결하고 선언적인 문법을 제공하여 **직관적이고 융통성과 확장성 있는 체인 구성**이 가능하다.
- LCEL은
  - 선형적 흐름 구조를 가진다.
  - 문법이 간결하고 선언적이다.
  - 체인의 구조가 코드만 봐도 쉽게 파악된다.
  - 유연하고 확장성이 매우 뛰어나다.
- `Runnable` 기반 구조
  - LCEL방식을 구성하는 모든 컴포넌트들은 `Runnable` 이라는 공통 인터페이스를 기반으로 동작한다.
  - 체인을 구성하는 각 컴포넌트들은 `Runnable` 을 상속하여 구현하여 이를 통해 일관된 실행 인터페이스를 제공한다.
  - **공통 메소드**:
    - `invoke()`: 단일 입력에 대한 처리
    - `batch()`: 다수 입력을 묶어서 한번에 처리
    - `stream()`: 스트리밍 방식의 요청
    - `ainvoke()`, `abatch()`, `astream()`: 비동기적 처리 메소드

In [1]:
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv()
prompt = ChatPromptTemplate.from_template(
    template="{item}에 어울리는 브랜드 이름 {count}개를 만들어 주세요"
)
model = ChatOpenAI(model="gpt-5-mini")
parser = StrOutputParser()

# input_variable -> (prompt) -> query -> (model) -> 답변 -> (parser) -> 최종답변

In [3]:
# 기존 off the shelf 방식

from langchain_classic import LLMChain
# chainㅏ을 구성하는 요소들을 넣어서 생성
chain = LLMChain( # prompt -> model -> parser 기본 체인을 구성
    prompt = prompt,
    llm=model,
    output_paser=parser
)
res = chain.invoke({"imtem":"가방", "count":3})
print(res)

C:\Users\USER\AppData\Local\Temp\ipykernel_22492\3067842342.py:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain( # prompt -> model -> parser 기본 체인을 구성


ValidationError: 1 validation error for LLMChain
output_paser
  Extra inputs are not permitted [type=extra_forbidden, input_value=StrOutputParser(), input_type=StrOutputParser]
    For further information visit https://errors.pydantic.dev/2.12/v/extra_forbidden

In [ ]:
print(res['text'])

In [2]:
# LCEL
chain2 = prompt | model | parser

print(type(chain2))

res2 = chain2.invoke({"item":"TV 브랜드", "count":3})
# 첫번째 구성요소 (prompt)에 전달 할 값을 전달.

<class 'langchain_core.runnables.base.RunnableSequence'>


In [3]:
print(res2)

좋습니다. TV 브랜드에 어울리는 이름 3가지(발음·의미 포함) 제안드립니다.

1. 빛마루 (Bitmaru)  
   - 의미: 빛(선명한 화질) + 마루(정상·공간). 따뜻하고 친근한 이미지의 화면 중심 브랜드.

2. 비전아크 (VisionArc)  
   - 의미: 비전(탁월한 시각 경험) + 아크(곡선·미래지향). 프리미엄·디자인을 강조하는 이름.

3. 클리어넥스 (ClearNex)  
   - 의미: Clear(선명함) + Next(차세대). 기술력과 혁신을 내세우는 모던한 느낌.

원하시면 각 이름에 맞는 로고 콘셉트나 슬로건도 함께 제안해 드리겠습니다. 상표·도메인 사용 가능 여부는 별도 확인을 권장합니다.


# Runnable 타입 주요 클래스

## [Runnable](https://reference.langchain.com/python/langchain_core/runnables/#langchain_core.runnables.base.Runnable)
- LangChain의 Runnable은 실행 가능한 작업 단위를 캡슐화한 개념으로, 데이터 흐름의 각 단계를 정의하고 **체인(chain) 에 포함 되어**  복잡한 작업의 각 단계를 수행 한다.
- Chain을 구성하는 class들은 Runnable의 상속 받아 구현한다.
- **Prompt Template클래스**, **Chat 모델, LLM 모델 클래스**, **Output Parser 클래스** 등 다양한 컴포넌트가 Runnable을 상속받아 구현된다.

### 주요 특징
- 작업 단위의 캡슐화:
    - Runnable은 특정 작업(예: 프롬프트 생성, LLM 호출, 출력 파싱 등)을 수행하는 독립적인 컴포넌트이다.
    - 각 컴포넌트는 독립적으로 테스트 및 재사용이 가능하며, 조합하여 복잡한 체인을 구성할 수 있다.
- 체인 연결 및 작업 흐름 관리:
    - Runnable은 체인(chain, 일련의 연결된 작업 흐름)을 구성하는 기본 단위로 사용된다.
    - LangChain Expression Language(LCEL)를 사용하면 | 연산자를 통해 여러 Runnable을 쉽게 연결할 수 있다.
    - 입력과 출력의 형식을 일관되게 유지하여 각 단계가 자연스럽게 연결된다.
- 모듈화 및 디버깅 용이성:
    - 각 단계가 명확히 분리되어 문제 발생 시 어느 단계에서 오류가 발생했는지 쉽게 확인할 수 있다.
    - 복잡한 작업을 작은 단위로 나누어 체계적으로 관리할 수 있다.
      
### Runnable의 표준 메소드
- 모든 Runnable이 구현하는 공통 메소드
    - **`invoke(input, config:RunnableConfig)->output`**: 단일 입력을 처리하여 결과를 반환.
    - **`batch(input:list, config:RunnableConfig|list[RunnableConfig]) -> list[Output]`**: 여러 입력 데이터들을 한 번에 처리.
    - **`stream(input, config:RunnableConfig) -> Iterator[Output]`**: 입력에 대해 스트리밍 방식으로 응답을 반환.
    - **`assign(**kwargs)`**:
      -  앞 Runnable의 출력 결과에 새로운 key–value 쌍의 Field 추가(assign) 하여 다음 Runnable로 전달.
      -  값으로는 Runnable 객체(LCEL체인등)나 고정 값(리터럴) 모두 가능하며, 각 항목은 실행 시 평가되어 기존 출력에 병합한다.
      -  주로 앞 단계의 출력에 부가 정보(field)를 추가하고자 할 때 사용한다. 특히 `RunnablePassthrough`와 결합해, 입력을 그대로 넘기면서 특정 field만 추가할 때 자주 사용
### Runnable의 주요 구현체(하위 클래스)

- **`RunnableSequence`**
    - 여러 `Runnable`을 순차적으로 연결하여 실행하는 구성이다.
    - 각 단계의 출력이 다음 단계의 입력으로 전달된다.
    - 보통은 LCEL 문법을 사용해서 정의한다.
      - LCEL을 사용하여 체인을 구성할 경우 자동으로 `RunnableSequence`로 변환된다.
  
-  **`RunnablePassthrough`**
    - 입력 데이터를 가공하지 않고 그대로 다음 단계로 전달하는 `Runnable`이다.
      - 앞 Runnable으로 부터 전달 받은 **입력 값을 다음 Runnable로 그대로 전달**한다.
           - `RunnablePassthrough()`
      - 입력받은 값에 **Field를 추가**해서 전달할 경우 `assign()` 메소드를 사용한다.
           - `RunnablePassthrough.assign(new_key1="new_value1", new_key2="new_value2", ..)`

- **`RunnableParallel`**
    - 여러 `Runnable`을 병렬로 실행한 후, 결과를 결합하여 다음 단계로 전달한다.
    - 
        ```python
        RunnableParallel(
            {
                "key1":Runnable1, 
                "key2":Runnable2,
                "key3":Runnable3, ...
            }
        )
        ```
      - 각 Runnable의 실행결과를 Value로 Dictionary를 생성해서 반환한다.
      - LCEL로 정의할 때는 Chain에 dictionary로 정의한다.

- **`RunnableLambda`**
    - 일반 함수를 `Runnable`로 변환할 때 사용한다.
    
    - Runnable을 입력해야 하는 자리에 함수를 넣어야 하는 경우 RunnableLambda로 그 함수를 Runnable로 만들어 넣는다.
    - **구현**
      1. `RunnableLambda(함수객체)`
      2. `@chain` decorator를 이용해 함수를 `RunnableLambda`로 구현할 수있다.
      3. LCEL 에 함수를 포함시키면 `RunnableLambda`로 자동 변환된다.

#### Runnable 예제

In [22]:
from langchain_core.runnables import Runnable

class MyRunnable(Runnable):
    # invoke(), stream(), batch(), assign(), 중에서 필요한 메소드를 재정의.
    def invoke(self, input_data, config=None):
        #input_data:처리를 위한 입력 값. 1개
        #config: RunnableConfig -> 입력시에는 dict
        #Runnable이 실행할 때 필요한 설정 정보를 입력.
        #  - {"configurable":{key:value}}
        output = ""
        if config is not None:
            if config("configurable",{}).get("lang"):
                output = "\n\n답변은 영어로 해주세요."
        return f"{input_data}에 대해서 한 문장으로 정의해주세요. {output}"


In [ ]:
mr1 = MyRunnable()
print(mr1.invoke("바나나"))
print(mr1.invoke("컴퓨터", config={"configurable":{"lang":"en"}}))

# 나중에 봐야겠다.

바나나에 대해서 한 문장으로 정의해주세요. 


TypeError: 'dict' object is not callable

In [ ]:
# chain

chain = mr1 | model
res = chain.invoke("사과", {"configurable":{"lang":"en"}})

#### RunnableLambda 예제

In [14]:
from langchain_core.runnables import RunnableLambda

mr2 = RunnableLambda(lambda input_data : f"{input_data}를 한 문장으로 설명해줘.")
isinstance(mr2, Runnable)

True

In [15]:
mr2.invoke("LLM")

'LLM를 한 문장으로 설명해줘.'

In [16]:
def sum(input_data):
    return input_data + 100

RunnableLambda(sum).invoke(1000)

1100

In [ ]:
# Runnable로 만들 함수
# 파라미터: 1개
# 반환값 : 다음 체인 구성에 전달할 값의 형식.

In [ ]:
prompt = ChatPromptTemplate.format_prompt(
    template = "{subject}에 대해 100 글자 이내로 설명해줘."
)
model = ChatOpenAI(modle="gpt-5-mini")
parser = StrOutputParser()

#출력결과, 글자수

chain = prompt | model | parser | RunnableLambda(lambda input_data:(input_data, len(input_data))) #튜플로 out

res = chain.invoke({"subject":"AI"})

TypeError: BaseChatPromptTemplate.format_prompt() missing 1 required positional argument: 'self'

In [ ]:
print(res)
len(res[0])

In [ ]:
def get_length(value):
    return value, len(value)

chain2 = prompt | model | parser | get_length 
# 이미 존재하는 함수라면 체인에 넣었을 때 자동으로 Runnable로 변환됨. 
# 람다식으로 넣으려면 RunnableLambda로 묶어야 함.
res = chain2.invoke("크리스마스")
res

#### RunnablePassThrough 예제

In [24]:
from langchain_core.runnables import RunnablePassthrough

rp = RunnablePassthrough()
res = rp.invoke("안녕하세요")
res = rp.invoke({"a":10, "b":20})
res = rp.invoke([1,2,30,40,10])

print(res)

[1, 2, 30, 40, 10]


In [25]:
# 입력 받은 값에 field를 추가해서 전달. 입력 : dict, item을 추가해서 전달.

address_runnable = RunnableLambda(lambda x: "서울시 금천구")
phone_runnable = RunnableLambda(lambda x : "010-4235-1234")

rp2 = RunnablePassthrough.assign(address = address_runnable, phone = phone_runnable)
 #변수 = Runnable(Callable) key:변수-value:Runnable 반환값을 input dict 에 추가

res = rp2.invoke({"name":"홍길동"})
res

{'name': '홍길동', 'address': '서울시 금천구', 'phone': '010-4235-1234'}

#### RunnableParallel 예제

In [28]:
from langchain_core.runnables import RunnableParallel

run1 = RunnableLambda(lambda x : x + 10)
run2 = RunnableLambda(lambda x : x * 10)
run3 = RunnableLambda(lambda x : x ** 10)

parallel = RunnableParallel(
    {
        "value1":run1, # run1의 실행결과를 value1에 담아서 반환.
        "value2":run2,
        "value3":run3,
        "value4":RunnablePassthrough() # value4 입력값
    }
)
res = parallel.invoke(5)
res

{'value1': 15, 'value2': 50, 'value3': 9765625, 'value4': 5}

In [ ]:
# chain = RunnableLambda(lambda x : x['v1']) | parallel
from operator import itemgetter
chain = itemgetter('v2') | parallel
# itemgetter('v1')({"v1":10, "v2":20}) itemgetter 가 체인 안에서 동작하는 방식

res = parallel.invoke({"v1":10, "v2":20})
res

{'value1': 15, 'value2': 50, 'value3': 9765625, 'value4': 5}

In [31]:
from operator import itemgetter

ig = itemgetter('v1') # 자료구조에서 "v1" 값을 조회

d = {"v1":10, "v2":20}
ig(d)

ig2 = itemgetter(2) # 자료구조[2] 조회한 결과를 반환.
ig2([1, 2, 3, 4])
ig2((10,20,30,40,50))
ig2({1:'a', 2:'c'})

'c'

In [32]:
# LCEL Chain 안에서 RunnableParellel 표현식, {key:Runnable1}
chain = run1 | {'result1':run2, 'result2':run3}
chain.invoke(20)

{'result1': 300, 'result2': 590490000000000}

In [ ]:
# {} 와 Runnable이 `|`로 묶이면 {}이 RunnableParellel 로 변환
chain2 = {'result1':run2, 'result2':run3} | RunnablePassthrough()
chain2.invoke(2)

{'result1': 20, 'result2': 1024}

In [34]:
rp = {'result1':run2, 'result2':run3}
rp.invoke(2)

AttributeError: 'dict' object has no attribute 'invoke'

### LCEL Chain 예제

In [ ]:
# 입력 : 음식이름
# 출력 : 음식의 레시피
# chain : prompt_template -> model(gpt-5-mini) -> StrOutParser

from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv()
prompt = ChatPromptTemplate.from_template(
    template="{food}의 레시피를 순서대로 알려줘." # 길게 ㅇ작성할 때에는 독스트링을 사용.
)
model = ChatOpenAI(model="gpt-5-mini")
parser = StrOutputParser() # 체인에 바로 붙이기도 함.

chain = prompt | model | parser

res = chain.invoke({"food":"핫케이크"})

In [37]:
print(res)

아래는 2–3인분(약 6장) 기준의 기본 핫케이크(팬케이크) 레시피입니다. 순서대로 따라 하세요.

재료
- 박력분(또는 중력분) 200g
- 설탕 2큰술(약 25g)
- 베이킹파우더 2작은술(약 8–10g)
- 소금 한 꼬집(약 1/4작은술)
- 달걀 1개
- 우유 250ml (또는 버터밀크 사용)
- 녹인 버터 또는 식용유 2큰술(약 30g)
- 식용유(팬 코팅용) 약간
- (선택) 바닐라 에센스 1/2작은술

만드는 법 (순서)
1. 마른 재료 체치기: 큰 볼에 밀가루, 설탕, 베이킹파우더, 소금을 넣고 골고루 섞은 뒤 체에 내려 공기 넣어둡니다.
2. 젖은 재료 혼합: 다른 볼에 달걀을 풀고 우유와 녹인 버터(또는 식용유), 바닐라를 넣어 섞습니다.
3. 반죽 합치기: 젖은 재료를 마른 재료에 한꺼번에 붓고 주걱으로 가볍게 섞습니다. 덩어리가 조금 남아 있어도 괜찮습니다(과도한 혼합은 식감을 떨어뜨립니다).
4. 반죽 휴지(선택) : 반죽을 10–15분 정도 실온에 두면 글루텐이 안정되고 더 폭신해집니다.
5. 팬 예열: 중약불로 팬을 예열한 뒤 키친타월에 기름을 묻혀 팬을 얇게 코팅합니다.
6. 굽기: 한 장당 약 60–80ml(종이컵 1/4 정도)씩 반죽을 부어 동그랗게 펴줍니다. 표면에 기포가 올라오고 가장자리가 살짝 굳으면(약 1.5–2분) 뒤집어 반대쪽도 1–2분 더 구워줍니다. 불이 너무 세면 겉만 타고 속이 익지 않으니 중약불 유지.
7. 보온 및 서빙: 다 구운 핫케이크는 100°C로 예열한 오븐에 두어 따뜻하게 유지하거나 접시에 쌓지 말고 겹치며 바로 서빙합니다. 버터, 메이플시럽, 과일, 생크림 등과 함께 내세요.

팁
- 우유 대신 버터밀크(또는 우유+식초·레몬즙 1큰술 섞어 5분 방치)를 쓰면 더 부드럽고 풍미가 좋아집니다.
- 반죽이 너무 묽으면 밀가루를 조금 더, 너무 되면 우유를 조금 추가하세요.
- 큰 팬으로 한 번에 여러 장 구울 때는 팬을 너무 가득 채우지 마세요(열 분포 방해).

필요하면 1인분·4인분용 분량으로 변경

In [40]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model="gpt-5-mini")
parser = StrOutputParser() # 체인에 바로 붙이기도 함.

prompt = ChatPromptTemplate.from_template( # 길게 작성할 때에는 독스트링을 사용.
  template= """ #Instruction
당신은 숙련된 요리 전문 assistant 입니다. 
요청한 음식의 레시피를 자세하게 작성해 주세요.

#Input Data
음식이름: {food}

#Output Indicator
- markdown 형식으로 작성한다.
- 다음 항목들을 넣어 레시피를 작성한다.
    -요리 이름
    -요리 기본 정보
        -난이도
        -조리시간
        -인분
    -재료
    -요리 방법
    -팁
"""
  )
  


recipe_chain = prompt | model | parser

In [41]:
resp = recipe_chain.invoke({"food":"핫케이크"})

In [42]:
print(resp)

# 요리 이름
핫케이크 (Hotcake / 팬케이크)

## 요리 기본 정보
- 난이도: 쉬움 — 중급(반죽 농도, 불 조절만 주의하면 누구나 가능)
- 조리시간: 총 약 25–35분 (준비 10분 + 반죽 휴지 10분(선택) + 조리 10–15분)
- 인분: 3–4인분 (지름 8–10cm 팬케이크 8장 기준)

## 재료
(기본 레시피 — 지름 약 8–10cm 팬케이크 8장 분량)
- 박력분(또는 중력분): 200g (약 1⅔컵)
- 설탕: 20–30g (2–3큰술) — 단맛 조절
- 베이킹파우더: 12g (약 2작은술)
- 소금: 1/4작은술
- 우유: 240ml (1컵)
- 계란: 1개 (M~L)
- 녹인 버터 또는 식용유: 30g (약 2큰술)
- 바닐라 익스트랙(선택): 1작은술
- 식용유 또는 무염버터(팬에 바를 용도): 약 1작은술씩

(토핑/옵션)
- 메이플 시럽, 꿀, 슈가파우더, 생크림, 버터, 과일(바나나, 베리류), 견과류 등

(플러피(폭신) 버전 — 더 폭신하게 만들고 싶을 때)
- 계란 2개로 분리: 노른자 1개 추가(위 기본과 동일) + 흰자 1개를 머랭으로 사용
- 크림 오브 타르타르 한 꼬집(흰자 안정용)

## 요리 방법
1. 재료 준비
   - 모든 재료는 상온(특히 우유와 계란)이면 섞이기 쉽고 결과가 더 부드럽습니다.
   - 박력분에 베이킹파우더, 설탕, 소금 등을 먼저 체에 내려 공기층을 만들어 줍니다.

2. 마른 재료 섞기
   - 큰 볼에 박력분 200g, 설탕 20–30g, 베이킹파우더 12g, 소금 1/4작은술을 넣고 골고루 섞어 체에 한 번 내립니다.

3. 젖은 재료 섞기
   - 다른 볼에 우유 240ml, 계란 1개, 녹인 버터(또는 식용유) 30g, 바닐라 1작은술을 넣고 잘 섞습니다.

4. 반죽 만들기
   - 젖은 재료를 마른 재료에 한 번에 붓고 주걱으로 가볍게 섞습니다.
   - 덩어리가 약간 남아 있는 정도(과도한 섞기는 글루텐 형성으로 질겨짐)를 목표로 10–15회 가볍게 저어주십시오.


In [44]:
resp2 = ""
for token in recipe_chain.stream({"food":"오일 파스타"}):
    print(token, end="")
    resp2 += token

# 오일 파스타 (Aglio e Olio 스타일)

## 요리 기본 정보
- 난이도: 쉬움
- 조리시간: 약 20–25분
- 인분: 2인분

## 재료
- 스파게티 면 200 g  
- 엑스트라 버진 올리브오일 40–60 ml (약 3–4큰술)  
- 마늘 4–6쪽 (편으로 얇게 썰기 또는 다지기)  
- 건 고추플레이크(또는 말린 고춧가루) 1/2–1작은술 (매운 정도에 따라 조절)  
- 소금(파스타 삶는 물용) 충분히 — 물 1L당 약 10 g 권장  
- 후춧가루 약간  
- 이탈리안 파슬리 약간(다져서 1–2큰술)  
- 파스타 삶은 물 120–180 ml (필요 시 추가)  
- 파르미지아노 레지아노(선택) 약 20–30 g (갈아서 마무리용)  
- 레몬 제스트(선택) 약 1작은술 — 상큼함을 원할 때  
- 앤초비 1–2필렛(선택) — 감칠맛을 원할 때  
- 빵가루(팬에 구워 토핑용, 선택) 2큰술

## 요리 방법
1. 물 끓이기  
   - 큰 냄비에 물 1.5–2L를 넣고 센 불에서 끓인다. 물이 팔팔 끓기 시작하면 소금을 넣는다(물 1L당 약 10 g 권장).  

2. 파스타 삶기  
   - 스파게티를 넣고 포장지에 적힌 시간보다 1분 적게(al dente 기준) 삶는다. 삶는 동안 면이 서로 붙지 않게 가끔 저어준다.  
   - 면이 다 삶기 1–2분 전에 삶은 물을 120–180 ml 정도 덜어 따로 보관한다(파스타 소스 조절용).

3. 마늘·향신료 준비 및 오일 가열  
   - 넉넉한 프라이팬(파스타를 버무릴 수 있는 크기)에 올리브오일을 붓고 중약불로 가열한다.  
   - 앤초비를 사용할 경우 오일에 넣고 숟가락으로 으깨며 녹여 감칠맛을 낸다(앤초비가 완전히 녹아 향만 남기면 됨).  
   - 마늘을 넣고 약한 불에서 천천히 향을 낸다. 마늘이 살짝 노릇해질 때까지(갈색이 되기 직전) 천천히 익힌다 — 과도하게 갈색이 나면 쓴맛이 생기므로 주의.  
   - 고추플레이크를 넣어 10–20초 더 향을 올린다.

4. 파

In [ ]:
# from textwrap import dedent

# template=dedent(""" #Instruction
#     당신은 숙련된 요리 전문 assistant 입니다. 
#     요청한 음식의 레시피를 자세하게 작성해 주세요.

#     #Input Data
#     음식이름: {food}

#     #Output Indicator
#     - markdown 형식으로 작성한다.
#     - 다음 항목들을 넣어 레시피를 작성한다.
#         -요리 이름
#         -요리 기본 정보
#             -난이도
#             -조리시간
#             -인분
#         -재료
#         -요리 방법
#         -팁
# """)
# print(template)

#Instruction
   당신은 숙련된 요리 전문 assistant 입니다. 
   요청한 음식의 레시피를 자세하게 작성해 주세요.

   #Input Data
   음식이름: {food}

   #Output Indicator
   - markdown 형식으로 작성한다.
   - 다음 항목들을 넣어 레시피를 작성한다.
       -요리 이름
       -요리 기본 정보
           -난이도
           -조리시간
           -인분
       -재료
       -요리 방법
       -팁



In [45]:
# ToDo
## 입력 : 번역할 내용, 언어
## 출력 : "번역할 내용"을 "언어"로 번역한 결과

prompt_trans = ChatPromptTemplate.from_template(
    template = """#Instruction
당신은 다국어가 가능ㅎ나 숙련된 번역 assistant 입니다.
요청된 문서의 내용을 지정된 언어로 번역해 주세요.
#Input Data
-번역할 내용:{content}
-번역할 언어:{lang}
"""
)

# 체인을 활용하면 이전에 설정한 모델과 파서등을 재사용 할 수 있다.

translate_chain = prompt_trans | model | parser

In [46]:
resp = translate_chain.invoke(
    {"content":"안녕하세요.", "lang":"불어"}
)

In [47]:
resp

'Bonjour.'

In [48]:
resp3 = translate_chain.invoke(
    {"content":resp2, "lang":"영어"}
)

In [49]:
print(resp3)

# Oil Pasta (Aglio e Olio style)

## Basic Info
- Difficulty: Easy
- Cooking time: About 20–25 minutes
- Servings: 2

## Ingredients
- Spaghetti 200 g  
- Extra virgin olive oil 40–60 ml (about 3–4 tablespoons)  
- Garlic 4–6 cloves (thinly sliced or minced)  
- Dried chili flakes (or ground dried chili) 1/2–1 teaspoon (adjust to taste)  
- Salt (for pasta water) — enough; about 10 g per 1 L of water recommended  
- Ground black pepper, to taste  
- Italian parsley, chopped, about 1–2 tablespoons  
- Pasta cooking water 120–180 ml (reserve more if needed)  
- Parmigiano-Reggiano (optional) about 20–30 g, grated for finishing  
- Lemon zest (optional) about 1 teaspoon — if you want brightness  
- Anchovy fillets 1–2 (optional) — for extra umami  
- Breadcrumbs (toasted in pan for topping, optional) 2 tablespoons

## Method
1. Bring the water to a boil  
   - Put 1.5–2 L of water in a large pot and bring to a rolling boil over high heat. When the water is boiling, add salt (about 10 g pe

### Chain과 Chain간의 연결

In [ ]:
type(recipe_chain) # RunnableSequence. chain에 넣을 수 있음.

langchain_core.runnables.base.RunnableSequence

In [ ]:
# recipe_chain - translate_chain (연결)
# 음식 레시피를 원하는 언어로 출력.
# recipe_chain: 입력 - food:음식이름
# translate_chain: 입력 - content:번역할 내용, lang:번역할 언어

In [54]:
# chain입력 : {food:음식이름, language:언어}
# 첫번째 chain: food
# language 는 두번째 chain으로 넘겨야 한다.
# 첫번째 chain 의 출력 결과를 content로 두번째 chain 으로 넘긴다.

chain = {
    "content":recipe_chain,
    "lang":itemgetter("lang")
} | translate_chain


result = chain.invoke({"food":"나폴리탄 파스타", "lang":"이태리어"})

In [55]:
print(result)

# Spaghetti alla Napolitana

## Informazioni di base
- Difficoltà: Facile (adatto anche ai principianti)  
- Tempo totale: circa 25 minuti (10 min preparazione / 15 min cottura)  
- Porzioni: 2

## Ingredienti
- Spaghetti 200 g  
- Acqua per la cottura della pasta circa 2 L  
- Sale per la cottura della pasta 1 cucchiaio  
- Olio d'oliva o olio vegetale 1 cucchiaino (un po' nell'acqua di cottura)  
- Cipolla 1/2 (tagliata a fettine sottili)  
- Peperone (rosso o giallo) 1/2 (tagliato a strisce lunghe) — oppure 1/2 peperone verde  
- Prosciutto o salsiccia 100 g (tagliati a strisce o a bocconcini)  
- Burro 1 cucchiaio  
- Ketchup 4 cucchiai (a piacere 3–5 cucchiai)  
- Salsa Worcestershire 1 cucchiaino (o poca salsa saporita)  
- Zucchero 1/2 cucchiaino (per smorzare l’acidità del ketchup)  
- Pepe q.b.  
- Sale q.b. (per aggiustare alla fine)  
- Prezzemolo tritato q.b. — per guarnire  
- Ingredienti opzionali: 1 spicchio d’aglio (tritato), 3–4 funghi champignon (a fette), formaggio (

In [ ]:
from IPython.display import Markdown
Markdown(result)

In [ ]:
recipe_chain
translate_chain
chain

## 사용자 함수를 Chain에 적용하기

### 사용자 함수를 Runnable로 정의 (RunnableLambda)
- 임의의 함수를 Runnable로 정의 할 수있다.
  - chain에 포함할 기능을 함수로 정의할 때 주로 사용. 
- `RunnableLambda(함수)` 사용
  - 함수는 invoke() 메소드를 통해 입력받은 값을 받을 **한개의 파라미터**를 선언해야 한다.
  - 보통 Lambda 표현식으로 정의한 함수를 LCEL chain에 포함시킬 때 사용한다.
  
#### Runnable 에 사용할 **사용자 정의 함수** 구문
- 이전 Chain의 출력을 입력 받는 **파라미터를 한개** 선언한다. (첫번째 파라미터)
- `invoke()`로 호출 할때 전달 하는 추가 설정을 입력받는 파라미터를 선언한다.(두번째 파라미터 - Optional)
  - RunnableConfig 타입의 값을 받는데 Dictionary 형식으로 `{"configuable": {"설정이름":"설정값"}}` 형식으로 받는다.
- 만약 함수가 여러개의 인자를 받는 경우 단일 입력을 받아들이고 이를 여러 인수로 풀어내는 래퍼 함수를 작성하여 Runnable로 만든다.
  ```python
  def plus(num1, num2):
      ...

  def wrapper_plus(nums:dict|list):
      return plus(nums['num1'], nums['num2'])
  ```

### 함수 자체를 chain에 추가
- RunnableLambda에 전달할 함수 구문에 맞는 함수라면 RunnableLambda를 사용하지 않고 chain에 넣을 수 있다. 
- 단 함수로 정의하고 LCEL에 포함시켜야 한다. Lambda 표현식으로 작성한 함수는 `RunnableLambda`를 사용해야 한다.

### 사용자 함수를 Chain으로 정의
- Chain 을 구성하는 작업 사이에 추가 작업이 필요할 경우, 중간 결과를 모두 사용해야 하는 경우 함수로 구현한다.
- `@chain` 데코레이터를 사용해 함수에 선언한다.

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

def plus(num1, num2):
    return num1 + num2

def wrapper_plus(num:list)->int:
    return plus(num[0], num[1])

chain = RunnablePassthrough() | wrapper_plus
chain = RunnablePassthrough() | RunnableLambda(lambda num:plus(num[0], num[1]))
chain.invoke([10, 20])

30

In [ ]:
from langchain_core.runnables import chain
# @chain:  복잡한 체인 구조를 정의한 함수를 Runnable로 만들 때 사용
# LCEL: 순차구조(흐름)만 정의가 가능.
# @chain 함수를 이용해서 Chain 구성: 조건문, 반복문을 이용해 chain의 흐름을 제어할 경우 : 이런 걸 lang graph 라고 하는구나

@chain
def coustom_chain(input_data:dict) -> dict[str, str]:
    # input_data: food, lang, kor_recipe:bool
    # kor_recipe: True 한국어 레시피와 language 로 번역된 레시피 반환
    #           : False: language 로 번역된 레시피 반환
    # 조건절 -> LCEL 로는 구현이 안됨.
    food = input_data['food']
    lang = input_data['lang']
    is_kor = input_data['kor_recipe']

    korean_recipe = recipe_chain.invoke({"food":food})
    result = translate_chain.invoke({"content":korean_recipe, "lang":lang})

    if is_kor:
        return (korean_recipe, result)
    else:
        return result
    
coustom_chain



RunnableLambda(coustom_chain)

In [60]:
result = coustom_chain.invoke(
    {"food":"김치전", "lang":"스패니쉬", "kor_recipe":True}
)

len(result)

2

In [61]:
result = coustom_chain.invoke(
    {"food":"김치전", "lang":"스패니쉬", "kor_recipe":False}
)

len(result)

4441

In [62]:
result

'Kimchijeon (Tortita de kimchi)\n\nInformación básica\n- Dificultad: Fácil\n- Tiempo de cocción: Preparación 10 min + Cocción 10–15 min = Total 20–25 min\n- Raciones: 2–3 personas\n\n---\n\nIngredientes\n- Kimchi muy fermentado (bien sazonado) 250–300 g (aprox. 1 1/2–2 tazas)\n- Jugo de kimchi (opcional) 1–2 cucharadas\n- Harina para frituras (o harina de trigo) 100 g (aprox. 3/4 taza)\n- Harina de arroz o fécula de patata 30 g (aprox. 2 cucharadas) — opcional para mayor crocancia\n- Agua 120–150 ml (aprox. 1/2–2/3 taza) — ajustar según el estado del kimchi\n- Huevo 1 (opcional, para ligar y aportar sabor)\n- Cebolleta/cebolla de verdeo 1 unidad (picada)\n- Cebolla 1/4 unidad (cortada en tiras finas, opcional)\n- Aceite de cocina (o aceite con un toque de aceite de sésamo) 2–3 cucharadas (por sartén)\n- Sal al gusto (para ajustar)\n- Azúcar 1/2 cucharadita (opcional, para equilibrar la acidez del kimchi)\n\nIngredientes opcionales\n- Mariscos (calamar, camarón, etc.) 70–100 g — para co

# Cache

- 응답 결과를 저장해서 같은 질문이 들어오면 LLM에 요청하지 않고 저장된 결과를 보여주도록 한다.
    - 처리속도와 비용을 절감할 수 있다.
    - 특히 chatbot같이 비슷한 질문을 하는 경우 유용하다.
- 저장 방식은 `메모리`, `sqlite` 등 다양한 방식을 지원한다.
  
    ```python
    set_llm_cache(Cache객체)
    ```

In [67]:
from langchain_community.cache import InMemoryCache, SQLiteCache # SQLiteCache 파이썬 내장 DB
from langchain_core.globals import set_llm_cache

# set_llm_cache(InMemoryCache()) # 대화 내역을 메모리에 저장.
set_llm_cache(SQLiteCache(database_path="cache.sqlite"))
# SQLite db 이용. 디비파일 경로를 지정 

prompt = ChatPromptTemplate(
    [
        ("system", "당신은 유능한 assistant입니다. 질문에 20단어 이내로 답해주세요."),
        ("user","{query}")
    ]
)
model = ChatOpenAI(model='gpt-5-mini')
chain = prompt | model | StrOutputParser()


In [68]:
chain.invoke({"query":"인공지능의 미래에 대해 알려줘"})

'AI는 자동화·맞춤화 확대, 일자리 변화, 윤리·안전·규제 강화, 인간과 협업 심화.'

In [65]:
chain.invoke({"query":"인공지능의 미래에 대해 알려줘"})

'자동화 증가, 윤리·규제 중요, 인간과 협업 확대.'